In [1]:
# import cv2
import os
import numpy as np

# === CONFIGURACIÓN ===
imagenes_dir = 'C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/jpeg_images/detections/arco_superior'
labels_dir = 'C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/jpeg_images/detections/labels'
salida_dir = 'C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/jpeg_images/detections/anotaciones_finales'
os.makedirs(salida_dir, exist_ok=True)

# Mapeo de los dientes
mapeo = {'11': 0, '12': 1, '13': 2, '14': 3, '15': 4, '16': 5, '17': 6, '18': 7,
         '21': 8, '22': 9, '23': 10, '24': 11, '25': 12, '26': 13, '27': 14, '28': 15}

# Función para obtener el ID del diente a partir del nombre
def nombre_to_id(nombre):
    return mapeo.get(nombre, -1)

# Función para guardar las anotaciones en formato YOLO
def guardar_anotaciones(imagen, etiquetas, contornos, nombre_imagen):
    with open(os.path.join(salida_dir, nombre_imagen.replace('.jpeg', '.txt')), 'w') as f:
        for idx, diente in enumerate(etiquetas):
            if diente is not None:
                x, y, w_box, h_box = cv2.boundingRect(contornos[idx])
                xc = (x + w_box / 2) / imagen.shape[1]
                yc = (y + h_box / 2) / imagen.shape[0]
                bw = w_box / imagen.shape[1]
                bh = h_box / imagen.shape[0]
                class_id = mapeo[diente]
                f.write(f"{class_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")
    print(f"💾 Anotaciones guardadas para {nombre_imagen}.")

# === CARGAR Y PROCESAR IMÁGENES ===
imagenes = [f for f in os.listdir(imagenes_dir) if f.endswith('.jpeg')]

# Configuración de umbral de área mínima
umbral_area_minima = 50  # Ajusta este valor según lo que necesites

# Recorrer cada imagen
for nombre_imagen in imagenes:
    img_path = os.path.join(imagenes_dir, nombre_imagen)
    img = cv2.imread(img_path)
    if img is None:
        print(f"❌ No se pudo cargar la imagen: {img_path}")
        continue

    h, w = img.shape[:2]
    
    # === CARGAR LA CAJA DEL ARCO ===
    label_path = os.path.join(labels_dir, nombre_imagen.replace('.jpeg', '.txt'))
    cajas = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                _, xc, yc, bw, bh = map(float, line.strip().split())
                x1 = int((xc - bw/2) * w)
                y1 = int((yc - bh/2) * h)
                x2 = int((xc + bw/2) * w)
                y2 = int((yc + bh/2) * h)
                cajas.append((x1, y1, x2, y2))
    else:
        print("⚠️ Archivo de etiquetas no encontrado.")
        continue

    # === RECORTAR REGIÓN DEL ARCO ===
    x1, y1, x2, y2 = cajas[0]
    img_corte = img[y1:y2, x1:x2]

    # === CONVERTIR A BLANCO Y NEGRO (ESCALA DE GRISES) ===
    gris = cv2.cvtColor(img_corte, cv2.COLOR_BGR2GRAY)

    # === MEJORAR RESOLUCIÓN (AJUSTE DE CONTRASTE Y BRILLO) ===
    gris_ecualizado = cv2.equalizeHist(gris)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gris_clahe = clahe.apply(gris)

    # === APLICAR FILTROS PARA PONER LOS PÍXELES MÁS ALTOS BLANCOS Y LOS MÁS BAJOS NEGROS ===
    _, gris_umbralizado = cv2.threshold(gris_clahe, 200, 255, cv2.THRESH_BINARY)

    # === DETECCIÓN DE CONTORNOS Y CIERRE MORFOLÓGICO ===
    bordes = cv2.Canny(gris_umbralizado, 50, 80)
    kernel = np.ones((1, 1), np.uint8)
    cerrado = cv2.morphologyEx(bordes, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Encontrar contornos
    contornos, _ = cv2.findContours(cerrado, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # === FILTRADO DE CONTORNOS POR ÁREA ===
    contornos_filtrados = []
    etiquetas_filtradas = []

    for idx, cnt in enumerate(contornos):
        area = cv2.contourArea(cnt)
        if area >= umbral_area_minima:
            contornos_filtrados.append(cnt)
            etiquetas_filtradas.append(None)  # Aún sin etiquetas

    # === FUNCIONES DE INTERFAZ ===
    def mostrar(img, contornos, etiquetas):
        copia = img.copy()
        for idx, cnt in enumerate(contornos):
            x, y, w, h = cv2.boundingRect(cnt)
            color = (0, 255, 0) if etiquetas[idx] is None else (0, 0, 255)
            # Dibujar una caja delimitadora en los contornos con un grosor más fino (1 píxel)
            cv2.rectangle(copia, (x, y), (x + w, y + h), color, 1)  # Grosor cambiado de 2 a 1
            # Mostrar el número del diente (sin la palabra "diente")
            if etiquetas[idx] is not None:
                cv2.putText(copia, f"{etiquetas[idx]}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        return copia

    # Función para manejar clic en la imagen y asignar etiquetas
    def mouse_callback(event, x, y, flags, param):
        global seleccion_idx
        if event == cv2.EVENT_LBUTTONDOWN:
            for idx, cnt in enumerate(contornos_filtrados):  # Usamos los contornos filtrados
                if cv2.pointPolygonTest(cnt, (x, y), False) >= 0:
                    seleccion_idx = idx
                    diente = input(f"🦷 Número del diente para contorno {idx + 1}: ").strip()
                    class_id = nombre_to_id(diente)
                    if class_id != -1:
                        etiquetas_filtradas[idx] = diente  # Asignar el diente al contorno
                        print(f"✅ Diente {diente} asignado.")
                    else:
                        print("⚠️ Número inválido.")
                    break

    # === INTERFAZ DE USUARIO PARA DIBUJAR Y ASIGNAR DIENTES ===
    cv2.namedWindow("Etiqueta dientes", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Etiqueta dientes", 1000, 800)
    cv2.setMouseCallback("Etiqueta dientes", mouse_callback)

    while True:
        vis = mostrar(img_corte, contornos_filtrados, etiquetas_filtradas)
        cv2.imshow("Etiqueta dientes", vis)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            # Guardar anotaciones para la imagen actual
            guardar_anotaciones(img, etiquetas_filtradas, contornos_filtrados, nombre_imagen)
            break
        elif key == ord('q'):
            print("⏩ Saliendo sin guardar.")
            break

    cv2.destroyAllWindows()


🦷 Número del diente para contorno 3:  22


✅ Diente 22 asignado.


🦷 Número del diente para contorno 2:  25


✅ Diente 25 asignado.


🦷 Número del diente para contorno 1:  q


...

✅ Diente 26 asignado.
💾 Anotaciones guardadas para image0169.jpeg.


🦷 Número del diente para contorno 1:  16


✅ Diente 16 asignado.


🦷 Número del diente para contorno 2:  26


✅ Diente 26 asignado.
💾 Anotaciones guardadas para image0170.jpeg.
💾 Anotaciones guardadas para image0171.jpeg.
💾 Anotaciones guardadas para image0172.jpeg.
💾 Anotaciones guardadas para image0173.jpeg.
💾 Anotaciones guardadas para image0174.jpeg.
💾 Anotaciones guardadas para image0175.jpeg.
💾 Anotaciones guardadas para image0176.jpeg.
💾 Anotaciones guardadas para image0177.jpeg.
💾 Anotaciones guardadas para image0178.jpeg.
💾 Anotaciones guardadas para image0179.jpeg.
💾 Anotaciones guardadas para image0180.jpeg.
💾 Anotaciones guardadas para image0181.jpeg.
💾 Anotaciones guardadas para image0182.jpeg.
💾 Anotaciones guardadas para image0183.jpeg.
💾 Anotaciones guardadas para image0184.jpeg.


In [6]:
# Crear carpeta si no existe
salida_vis_dir = 'C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/jpeg_images/detections/contorno dientes'
os.makedirs(salida_vis_dir, exist_ok=True)

# Dibujar y guardar imagen con contornos y etiquetas
img_etiquetada = mostrar(img_corte, contornos_filtrados, etiquetas_filtradas)
ruta_guardado_img = os.path.join(salida_vis_dir, nombre_imagen)
cv2.imwrite(ruta_guardado_img, img_etiquetada)
print(f"🖼️ Imagen anotada guardada en: {ruta_guardado_img}")

NameError: name 'mostrar' is not defined